In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from konlpy.tag import Okt
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./src/네이버종토방댓글_카카오_7500.csv')
df = df[df['댓글'] != 'error'] # 크롤링 에러 제거 (4개)
df = df.dropna() # 결측치 제거 (3개)
df

,날짜,댓글,조회수,좋아요,싫어요
0,2022-04-29,오늘은 아쉽지만 종가 예상 9만-9만1천...,7,0,0
1,2022-04-29,카카오 방이에요 ~!,9,0,0
2,2022-04-29,하방 경직성,21,1,1
3,2022-04-29,15만간다고 입방정떨때부터 알아봤다..,17,1,0
4,2022-04-29,역시 개 지 랄은 개카오..,32,2,0
...,...,...,...,...,...
149995,2021-02-10,"저 이제 시작한 주린이 인데,,,",898,2,0
149996,2021-02-10,답글:[삭제된 게시물의 답글]나는 익절한다,157,0,0
149997,2021-02-10,카카오도 공매가 있을것인데,228,1,0
149998,2021-02-10,카카오 목표가 585천원 써놓고,297,3,0


In [3]:
df['한글댓글'] = df['댓글'].str.replace('\[삭제된 게시물의 답글\]',' ')
df['한글댓글'] = df['한글댓글'].str.replace('답글:',' ')
df['한글댓글'] = df['한글댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
df = df[df['한글댓글'] != ''] # 공백제거
df = df.reset_index(drop=True)
df

,날짜,댓글,조회수,좋아요,싫어요,한글댓글
0,2022-04-29,오늘은 아쉽지만 종가 예상 9만-9만1천...,7,0,0,오늘은 아쉽지만 종가 예상 만 만 천
1,2022-04-29,카카오 방이에요 ~!,9,0,0,카카오 방이에요
2,2022-04-29,하방 경직성,21,1,1,하방 경직성
3,2022-04-29,15만간다고 입방정떨때부터 알아봤다..,17,1,0,만간다고 입방정떨때부터 알아봤다
4,2022-04-29,역시 개 지 랄은 개카오..,32,2,0,역시 개 지 랄은 개카오
...,...,...,...,...,...,...
147840,2021-02-10,"저 이제 시작한 주린이 인데,,,",898,2,0,저 이제 시작한 주린이 인데
147841,2021-02-10,답글:[삭제된 게시물의 답글]나는 익절한다,157,0,0,나는 익절한다
147842,2021-02-10,카카오도 공매가 있을것인데,228,1,0,카카오도 공매가 있을것인데
147843,2021-02-10,카카오 목표가 585천원 써놓고,297,3,0,카카오 목표가 천원 써놓고


In [4]:
okt = Okt()
okt3_list = []
for i in range(df.shape[0]):
    comment = df['한글댓글'][i]
    n_list = okt.morphs(comment, stem=True)
    okt3_list += n_list
    if i % 10000 == 0:
        print(f'현재{i}개 형태소 분리 완료')

현재0개 형태소 분리 완료
현재10000개 형태소 분리 완료
현재20000개 형태소 분리 완료
현재30000개 형태소 분리 완료
현재40000개 형태소 분리 완료
현재50000개 형태소 분리 완료
현재60000개 형태소 분리 완료
현재70000개 형태소 분리 완료
현재80000개 형태소 분리 완료
현재90000개 형태소 분리 완료
현재100000개 형태소 분리 완료
현재110000개 형태소 분리 완료
현재120000개 형태소 분리 완료
현재130000개 형태소 분리 완료
현재140000개 형태소 분리 완료


In [5]:
df_sam = pd.read_csv('./src/네이버종토방댓글_삼성전자_10000.csv')
df_sam = df_sam[df_sam['댓글'] != 'error'] # 크롤링 에러 제거 
df_sam = df_sam.dropna() # 결측치 제거 
df_sam

,날짜,댓글,조회수,좋아요,싫어요
0,2022-05-02,개미들 바닥에서 잘샀네..,29,0,2
1,2022-05-02,* 내일 상한가 : 코스모화학...,17,0,0
2,2022-05-02,문재앙이 가장 이해 안가는일 양산 아방궁...,29,4,3
3,2022-05-02,"●[속보]문재인,이재용 사면 안한다●",50,0,0
4,2022-05-02,사면 나가리~,29,1,0
...,...,...,...,...,...
199995,2021-12-14,양성전자가자!! 10만가자!!,325,0,0
199996,2021-12-14,답글:[삭제된 게시물의 답글]쳐맞고 있는건 석...,294,0,1
199997,2021-12-14,규모 4.9 지진,345,2,3
199998,2021-12-14,지진났어도 뭉가리는호주에서 신나게ㅡ,332,3,3


In [6]:
df_sam['한글댓글'] = df_sam['댓글'].str.replace('\[삭제된 게시물의 답글\]',' ')
df_sam['한글댓글'] = df_sam['한글댓글'].str.replace('답글:',' ')
df_sam['한글댓글'] = df_sam['한글댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
df_sam = df_sam[df_sam['한글댓글'] != ''] # 공백제거
df_sam = df_sam.reset_index(drop=True)
df_sam

,날짜,댓글,조회수,좋아요,싫어요,한글댓글
0,2022-05-02,개미들 바닥에서 잘샀네..,29,0,2,개미들 바닥에서 잘샀네
1,2022-05-02,* 내일 상한가 : 코스모화학...,17,0,0,내일 상한가 코스모화학
2,2022-05-02,문재앙이 가장 이해 안가는일 양산 아방궁...,29,4,3,문재앙이 가장 이해 안가는일 양산 아방궁
3,2022-05-02,"●[속보]문재인,이재용 사면 안한다●",50,0,0,속보 문재인 이재용 사면 안한다
4,2022-05-02,사면 나가리~,29,1,0,사면 나가리
...,...,...,...,...,...,...
198005,2021-12-14,양성전자가자!! 10만가자!!,325,0,0,양성전자가자 만가자
198006,2021-12-14,답글:[삭제된 게시물의 답글]쳐맞고 있는건 석...,294,0,1,쳐맞고 있는건 석
198007,2021-12-14,규모 4.9 지진,345,2,3,규모 지진
198008,2021-12-14,지진났어도 뭉가리는호주에서 신나게ㅡ,332,3,3,지진났어도 뭉가리는호주에서 신나게


In [7]:
okt = Okt()
oktsam_list = []
for i in range(df_sam.shape[0]):
    comment = df_sam['한글댓글'][i]
    n_list = okt.morphs(comment, stem=True)
    oktsam_list += n_list
    if i % 10000 == 0:
        print(f'현재{i}개 형태소 분리 완료')

현재0개 형태소 분리 완료
현재10000개 형태소 분리 완료
현재20000개 형태소 분리 완료
현재30000개 형태소 분리 완료
현재40000개 형태소 분리 완료
현재50000개 형태소 분리 완료
현재60000개 형태소 분리 완료
현재70000개 형태소 분리 완료
현재80000개 형태소 분리 완료
현재90000개 형태소 분리 완료
현재100000개 형태소 분리 완료
현재110000개 형태소 분리 완료
현재120000개 형태소 분리 완료
현재130000개 형태소 분리 완료
현재140000개 형태소 분리 완료
현재150000개 형태소 분리 완료
현재160000개 형태소 분리 완료
현재170000개 형태소 분리 완료
현재180000개 형태소 분리 완료
현재190000개 형태소 분리 완료


In [8]:
full_list = okt3_list+oktsam_list
print(Counter(full_list))

Counter({'이': 41545, '하다': 36409, '들': 26400, '가': 22760, '카카오': 22629, '은': 21683, '에': 21659, '는': 15694, '만': 13127, '의': 11991, '도': 10643, '보다': 10182, '오늘': 9934, '을': 9810, '아': 9774, '다': 9767, '되다': 8122, '이다': 7687, '만원': 7498, '주식': 6986, '윤석열': 6796, '개미': 6710, '전자': 6685, '이재명': 6626, '한': 6293, '주': 6167, '가다': 6104, '있다': 6044, '삼성': 5959, '를': 5789, '없다': 5754, '으로': 5752, '로': 5634, '삼전': 5581, '오다': 5510, '원': 5491, '들다': 5464, '안': 5297, '에서': 5080, '고': 5064, '개': 5058, '매수': 4867, '오르다': 4821, '대': 4617, '대통령': 4582, '나': 4437, '내': 4431, '아니다': 4418, '하': 4333, '왜': 4196, '라': 4153, '일': 4143, '공매도': 3974, '야': 3835, '지금': 3789, '국민': 3630, '까지': 3623, '년': 3621, '네': 3599, '때': 3522, '시': 3488, '내일': 3444, '전': 3436, '못': 3408, '중': 3377, '주가': 3353, '자': 3308, '민주당': 3281, '사람': 3243, '윤': 3179, '문재인': 3128, '같다': 3115, '좋다': 3112, '문': 3085, '더': 3045, '떨어지다': 2995, '요': 2986, '지': 2973, '말': 2968, '적': 2947, '놈': 2936, '과': 2935, '속보': 2906, '미국': 2863, '니': 

In [27]:
greed_word = ['매수','사','사다','사라','사면','사고','줍다','들어오다','들어가다','타다','수급','매집','올라타다' # 주식 구매 단어
              ,'탑승','불나방','담다','추매','추미애','영차'
              ,'오르다','올라가다','올리다','올려주다','올린다','오름','올려놓다','오른','상향' # 주식 가격 상승 단어
              ,'양봉','상방','상승','살아나다','양전','상한','반등','폭등','퍽등','급등'
              ,'탐욕','찬티','좋다','간다','가다','가즈','싸다','익절','제발','최고','돌파','수익','위대하다','먹다' # 탐욕 단어
              ,'기회','바닥','호재','버티다','존버','감사','감사하다','대박','대단하다','승리','찬양','믿다','흑우'
              ,'개꿀','회복','갓','홀딩','부활']

fear_word = ['공매도','공매','매도','팔','파다','팔다','팔고','팔면','던지다','털다','탈출','튀다','튀어','설거지' # 주식 판매 단어
             ,'손절','버리다'
             ,'떨어지다','떨구다','빠지다','하락','폭락','떡락','조정','급락','음봉','하방','폭포수','음전' # 주식 가격 하락 단어
             ,'반토막','내리다','내려오다','깨지다','대퍽락','나락','붕괴','추락'
             ,'공포','안티','망하다','물리다','끝나다','손해','폭망','거품','무섭다','자살','악재','상폐','개미지옥' # 공포 단어
             ,'시발','염병','욕','짜증나다','걸레','어휴','개','놈','아가리','빡치다','지랄','손실'
             ,'개관','주가조작','쓰레기','죽다','패닉']

####################################################################################################################################

neutral_word = ['카카오','주식','윤석열','개미','전자','이재명','삼성','삼전','대통령','국민','주가','민주당','문재인','어제'
                ,'속보','미국','한국','기업','전쟁','정치','주주','네이버','정권','죄명','나라','돈','만원','내일','오늘','사람'
                ,'러시아','코스피','재앙','대장동','윤석렬','프로','총','종가','대선','시작','중국','정부','상장','좌파','코로나'
                ,'안철수','평단','나스닥','증시','후보','김건희','대한민국','토론','투자','분할','시간','뉴스','우크라이나','당선'
                ,'회사','종목','국정원','낼','하나','개인','검찰','실적','청와대','보고','예상','적정','앵두','외인','기관'
                ,'외국인','세력','규제','상','물','배당','연기금','지지','금리','시장','선물','만주','엔','김범수','푸틴','북한'
                ,'거래','반도체','선거','차트','딸','허경영','투표','전라도','세계','가격','지수','말씀','끝','배당금','빨갱이'
                ,'힘','방','앞','수완','소리','목표','범수','다음','카','보수','단타','여론조작']

stop_word = ['이','하다','들','가','은','에','는','만','의','도','을','아','다','되다','이다','한','주','있다','명','층'
             ,'를','없다','으로','로','안','에서','고','나','내','네','때','시','전','못','중','자','수','두','성','잘','제'
             ,'보다','가다','오다','원','들다','대','아니다','하','왜','라','게','여기','하고','알다','대다','이네','님','형','점'
             ,'일','야','지금','까지','년','같다','문','더','요','지','말','적','과','니','위','애','다시','너','날','글','그렇다'
             ,'좀','이유','거','이제','진짜','조','뭐','삼','면','인','크다','장','억','월','부터','냐','인데','것','너무','이번'
             ,'오','생각','해','우리','돼다','우','아직도','이렇다','아직','에게','석','외','역시','나다','분','그','서','당','와'
             ,'저','재','기','또','나오다','잡다','매','맞다','주다','많다','살다','자다','윤','난','후','모르다','많이','한테','차'
             ,'받다','않다','치다','넘다','찍다','때문','간','여','이상','열','공','시키다','액','누가','언제','그냥','이나','내다'
             ,'밉다','지다','찢다','끄다','갈다','안되다','단','선','검','계속','추다','인들','랑','박','주리다','개다','양','구'
             ,'한번','차다','번','세','미치다','쓰다','만들다','뽑다','빨리','말다','이야','늘다','짜다','어떻다','빨','천','앞']

full_stop = greed_word + fear_word + stop_word + neutral_word

In [28]:
full_dict = dict(Counter(full_list))

for i in full_dict:
    if i in full_stop:
        full_dict[i] = 0
        
print(Counter(full_dict))

Counter({'인가': 866, '임': 865, '해보다': 859, '기다리다': 842, '처': 840, '뭔': 828, '해주다': 828, '보': 815, '무조건': 798, '배': 798, '얼마': 797, '딱': 792, '나가다': 791, '문제': 790, '무슨': 790, '월요일': 780, '솔직하다': 774, '손': 771, '빼다': 771, '교체': 763, '정도': 763, '라고': 760, '이냐': 755, '식': 750, '곧': 750, '코인': 742, '이렇게': 732, '경제': 727, '알': 726, '경': 723, '어디': 719, '마지막': 719, '범': 714, '급': 713, '종': 713, '몇': 708, '인간': 701, '어': 698, '반': 697, '검사': 694, '이준석': 693, '일본': 692, '달': 690, '긴급': 689, '알바': 687, '부동산': 687, '국': 685, '처럼': 685, '지지율': 685, '더불다': 682, '집': 679, '박근혜': 677, '걸': 675, '뜨다': 675, '들이다': 673, '국가': 673, '경북': 673, '말고': 671, '지지자': 671, '깨': 669, '시외': 666, '분석': 664, '만에': 662, '유영': 658, '함': 658, '답': 657, '불': 652, '싶다': 652, '두다': 649, '물량': 648, '근데': 642, '하루': 641, '같이': 641, '요즘': 639, '올해': 639, '걱정': 639, '가지': 638, '조작': 638, '정말': 637, '대구': 636, '대표': 632, '발': 632, '도리': 630, '순': 628, '여러분': 626, '이재용': 626, '수사': 626, '얘기': 624, '타격': 623, '니까': 620, '바': 620